In [ ]:
#PROCESS 
# install dcm2niix
# pip install dcm2bids
# dcm2bids_helper -d /Users/sinakling/disks/meso_shared/pRF_am/sub-001 -o /Users/sinakling/disks/meso_shared/pRF_am

#create config file (one aquisituon = 1 description, for each task and fieldmap)

In [2]:
import os
import shutil
import warnings
import json
import glob
import nibabel as nb
import json
warnings.filterwarnings('ignore')

In [ ]:
# Load main settings parameters
with open('settings.json') as f:
    json_s = f.read()
    settings = json.loads(json_s)

subjects = settings['subjects']
sessions = settings['sessions']
scratch_dir = settings['data_dir']
disk_dir = "/Users/sinakling/disks/meso_shared"

bids_dir = "{}/{}".format(disk_dir, settings['project_name'])
deriv_dir = "{}/derivatives".format(bids_dir)
source_dir = "{}/sourcedata".format(bids_dir)

output_dir = os.path.join(deriv_dir, "log_outputs")
os.makedirs(output_dir, exist_ok=True)
output_sh = os.path.join(output_dir, "create_folders.sh")

In [8]:
print(bids_dir)
print(deriv_dir)
print(source_dir)

/Users/sinakling/disks/meso_shared/pRF_am
/Users/sinakling/disks/meso_shared/pRF_am/derivatives
/Users/sinakling/disks/meso_shared/pRF_am/sourcedata


In [ ]:
SUBJECT_SESSION_MAP = {
    "sub-001": {
        "ses-01": "ses-rsprf",
        "ses-02": "ses-prfoccl",
        "ses-03": "ses-rs2",
        "ses-04": "ses-prfstrab",
        "ses-05": "ses-prfaniso",
        "ses-06": "ses-rs3",
    },
    "sub-002": {
        "ses-01": "ses-prfstrab",
        "ses-02": "ses-rs",
        "ses-03": "ses-rscalprf",
        "ses-04": "ses-prfoccl",
        "ses-05": "ses-prfaniso",
        "ses-06": "ses-prfaniso",
        "ses-07": "ses-rs3"
    },
    "sub-003": {
        "ses-01": "ses-prf_strab",
        "ses-02": "ses-rs",
        "ses-03": "ses-prfaniso",
        "ses-04": "ses-rs2",
        "ses-05": "ses_occl",
        "ses-06": "ses-rs3",
    }
}

project_dir = os.path.join(settings['data_dir'], settings['project_name'])
sourcedata_dir = os.path.join(project_dir, "sourcedata")
os.makedirs(sourcedata_dir, exist_ok=True)

for sub in subjects:
    print(f"\nProcessing {sub}")

    sub_dir = os.path.join(project_dir, sub)
    source_backup = os.path.join(sourcedata_dir, sub)

    # 1) Copy full original subject folder to sourcedata (if not already there)
    if not os.path.exists(source_backup):
        print(f"→ Backing up {sub_dir} → {source_backup}")
        shutil.copytree(sub_dir, source_backup, dirs_exist_ok=True)
    else:
        print(f"Backup already exists for {sub}, skipping copy")

    session_map = SUBJECT_SESSION_MAP[sub]

    for new_name, old_name in session_map.items():
        old_path = os.path.join(sub_dir, old_name)
        new_path = os.path.join(sub_dir, new_name)

        if not os.path.exists(old_path):
            print(f"{old_name} not found, skipping.")
            continue

        if os.path.exists(new_path):
            print(f"{new_name} already exists, skipping rename.")
            continue

        print(f"  Renaming {old_name} → {new_name}")
        os.rename(old_path, new_path)

    print("\Backup and session renaming complete.")
    





In [ ]:
#Step 2 
# copy event files 
# event files CAL: from https://github.com/mszinte/deepmreyecalib/tree/amsterdam-24/experiment_code/data/sub-01/ses-01/func
# event files pRF states: from https://github.com/mszinte/pRFexp/tree/pRFexp_altVision/data/sub-01

# rename files to match sessions

In [ ]:
#Step 3 create json files 

#TODO check if correct 

bold_json = {   "Modality": "MR",
                "MagneticFieldStrength": 7,
                "ImagingFrequency": 298.03,
                "Manufacturer": "Philips",
                "ManufacturersModelName": "Achieva",
                "InstitutionName": "Spinoza_Centre",
                "InstitutionalDepartmentName": "Spinoza_Centre",
                "InstitutionAddress": "Amsterdam",
                "DeviceSerialNumber": "00076",
                "StationName": "7t-acq-multix",
                "SeriesInstanceUID": "1.3.46.670589.11.76.5.0.6388.2022071316500147307",
                "StudyInstanceUID": "1.3.46.670589.11.76.5.0.7360.2022071316280508000",
                "StudyID": "694967284",
                "PatientPosition": "HFS",
                "SoftwareVersions": "5.1.7_5.1.7.0",
                "MRAcquisitionType": "3D",
                "ScanningSequence": "GR",
                "SequenceVariant": "SK",
                "ScanOptions": "FS",
                "ImageType": ["ORIGINAL", "PRIMARY", "PHASE", "MAP", "P", "FFE"],
                "SeriesNumber": 601,
                "PhilipsRWVSlope": 1.53455,
                "PhilipsRWVIntercept": -3142,
                "PhilipsRescaleSlope": 1.53455,
                "PhilipsRescaleIntercept": -3142,
                "PhilipsScaleSlope": 651.74,
                "UsePhilipsFloatNotDisplayScaling": 1,
                "SliceThickness": 1.8,
                "SpacingBetweenSlices": 1.8,
                "SAR": 1.96128,
                "EchoTime": 0.016901,
                "RepetitionTime": 0.0438647,
                "FlipAngle": 13,
                "CoilString": "MULTI_COIL",
                "PercentPhaseFOV": 100,
                "EchoTrainLength": 43,
                "PhaseEncodingSteps": 112,
                "AcquisitionMatrixPE": 112,
                "ReconMatrixPE": 112,
                "PixelBandwidth": 1762,
                "NumberOfVolumesDiscardedByUser": 3,
                "RepetitionTime": 1.33,
                "PhaseEncodingDirection": "j"
            }

topup_json = {  "Modality": "MR",
                "MagneticFieldStrength": 7,
                "ImagingFrequency": 298.03,
                "Manufacturer": "Philips",
                "ManufacturersModelName": "Achieva",
                "InstitutionName": "Spinoza_Centre",
                "InstitutionalDepartmentName": "Spinoza_Centre",
                "InstitutionAddress": "Amsterdam",
                "DeviceSerialNumber": "00076",
                "StationName": "7t-acq-multix",
                "SeriesInstanceUID": "1.3.46.670589.11.76.5.0.6388.2022071316500147307",
                "StudyInstanceUID": "1.3.46.670589.11.76.5.0.7360.2022071316280508000",
                "StudyID": "694967284",
                "PatientPosition": "HFS",
                "SoftwareVersions": "5.1.7_5.1.7.0",
                "MRAcquisitionType": "3D",
                "ScanningSequence": "GR",
                "SequenceVariant": "SK",
                "ScanOptions": "FS",
                "ImageType": ["ORIGINAL", "PRIMARY", "PHASE", "MAP", "P", "FFE"],
                "SeriesNumber": 601,
                "PhilipsRWVSlope": 2.04005,
                "PhilipsRWVIntercept": 0,
                "PhilipsRescaleSlope": 2.04005,
                "PhilipsRescaleIntercept": 0,
                "PhilipsScaleSlope": 7.09759e-05,
                "UsePhilipsFloatNotDisplayScaling": 1,
                "SliceThickness": 1.8,
                "SpacingBetweenSlices": 1.8,
                "SAR": 2.24486,
                "EchoTime": 0.0169,
                "RepetitionTime": 0.0438394,
                "FlipAngle": 13,
                "CoilString": "MULTI_COIL",
                "PercentPhaseFOV": 100,
                "EchoTrainLength": 43,
                "PhaseEncodingSteps": 112,
                "AcquisitionMatrixPE": 112,
                "ReconMatrixPE": 112,
                "PixelBandwidth": 1714,
                "NumberOfVolumesDiscardedByUser": 3,
                "RepetitionTime": 1.33,
                "PhaseEncodingDirection": "j-",
                "TotalReadoutTime": 0.0322
             }

In [ ]:
# create bold json files
import os
import glob
import json

# Mapping of task names to their label for BIDS JSON
TASK_MAP = {
    "CAL": "CAL",
    "pRF": "pRF",
    "pRFaniso": "pRFaniso",
    "pRFoccl": "pRFoccl",
    "pRFstrab": "pRFstrab",
    "RSC": "RSC",
    "RSL": "RSL",
    "RSR": "RSR"
}

for subject in subjects:
    for session in sessions:
        func_dir = f"{bids_dir}/{subject}/{session}/func"
        fmap_dir = f"{bids_dir}/{subject}/{session}/fmap"
        bold_files = glob.glob(f"{func_dir}/*_bold.nii.gz")

        # --- create BOLD jsons ---
        for bold_file in bold_files:
            json_file = f"{bold_file[:-7]}.json"
            bold_json_val = bold_json.copy()  # prevent overwriting the template

            # determine task name based on filename substring
            matched_task = None
            for key, task_name in TASK_MAP.items():
                if key in bold_file:
                    matched_task = task_name
                    break

            if matched_task:
                bold_json_val["TaskName"] = matched_task
            else:
                bold_json_val["TaskName"] = "unknown"

            with open(json_file, "w", encoding="utf-8", newline="\r\n") as fp:
                json.dump(bold_json_val, fp, indent=4, sort_keys=True, ensure_ascii=False)

        # --- create fmap jsons ---
        fmap_files = glob.glob(f"{fmap_dir}/*_epi.nii.gz")
        if not fmap_files:
            print(f"No fmap found for {subject} {session}")
            continue

        fmap_file = fmap_files[0]
        json_file = f"{fmap_file[:-7]}.json"
        topup_json_new = topup_json.copy()

        # IntendedFor = relative paths of bold files
        intended_for = [
            os.path.relpath(bold_file, start=bids_dir)
            for bold_file in bold_files
        ]
        topup_json_new["IntendedFor"] = intended_for

        with open(json_file, "w", encoding="utf-8", newline="\r\n") as fp:
            json.dump(topup_json_new, fp, indent=4, sort_keys=True, ensure_ascii=False)


In [ ]:
# Step 4 Create task .json files and make participant.tsv and .json manually 
# copy from retinomaps and deepmreyecalib